In [74]:
import numpy as np
from utils import *
import random

In [75]:
data = open('dinos.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))
# print(data)
print(chars)

There are 19909 total characters and 27 unique characters in your data.
['m', 'h', 'n', 'p', 'u', 'y', 'q', 'r', 'k', 'g', '\n', 'd', 's', 'x', 'v', 'o', 'e', 'i', 'b', 'f', 't', 'c', 'l', 'w', 'j', 'a', 'z']


In [76]:
char_to_ix = { ch:i for i, ch in enumerate(sorted(chars))}
ix_to_char = { i:ch for i, ch in enumerate(sorted(chars))}
# print(char_to_ix)
print(ix_to_char)
print(char_to_ix)


{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [77]:
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    clipped_gradients = {}
    for g in ['dWaa', 'dWax', 'dWya', 'db', 'dby']:
        clipped_gradients[g] = np.clip(gradients[g], -maxValue, maxValue)

    return clipped_gradients


In [78]:
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10

gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
# print('gradients_first = ', gradients)

gradients = clip(gradients, 10)

print('gradients = ', gradients)
print('gradients[\'dWaa\'][1][2] =', gradients['dWaa'][1][2])
print('gradients[\'dWax\'][3][1] =', gradients['dWax'][3][1])
print('gradients[\'dWya\'][1][2] =', gradients['dWya'][1][2])
print('gradients[\'db\'][4] =', gradients['db'][4])
print('gradients[\'dby\'][1] =', gradients['dby'][1])

gradients =  {'dWaa': array([[ -4.04677415,  -5.45359948, -10.        ,   9.82367434,
        -10.        ],
       [-10.        ,  -2.05649899,  10.        ,   2.36716267,
        -10.        ],
       [ -7.129932  ,   6.25244966,  -1.60513363,  -7.6883635 ,
         -2.30030722],
       [  7.45056266,  10.        , -10.        ,  -6.26416911,
         -8.03766095],
       [-10.        ,  -9.23792022, -10.        ,  10.        ,
         -1.31914233]]), 'dWax': array([[ 10.        ,   4.36509851,   0.96497468],
       [-10.        ,  -2.77388203,  -3.54758979],
       [ -0.82741481,  -6.27000677,  -0.43818169],
       [ -4.7721803 , -10.        ,   8.8462238 ],
       [  8.81318042,  10.        ,   0.50033642]]), 'dWya': array([[-10.        ,   6.46675452,  -3.56270759, -10.        ,
         -5.96649642],
       [ -5.8859438 ,  -8.73882298,   0.29713815, -10.        ,
         -2.67761865]]), 'db': array([[10.        ],
       [ 8.52797841],
       [10.        ],
       [10.        ]

In [79]:
# GRADED FUNCTION: sample

def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    ### START CODE HERE ###
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size, 1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a, 1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugging and prevents entering an infinite loop. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        # for grading purposes
        np.random.seed(counter+seed) 
#         print('y = ', y)
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        idx = np.random.choice(range(vocab_size), p=y.ravel())
        
        print('idx = ', idx)
        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input character as the one corresponding to the sampled index.
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
#         print('x = ', x)
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        
    ### END CODE HERE ###

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices 

In [80]:
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


# print(parameters)
# print(y)
indices  = sample(parameters, char_to_ix, 0)
# print(counter)
print("Sampling:")
print("list of sampled indices:", indices)
print("list of sampled characters:", [ix_to_char[i] for i in indices])

idx =  12
idx =  17
idx =  24
idx =  14
idx =  13
idx =  9
idx =  10
idx =  22
idx =  24
idx =  6
idx =  13
idx =  11
idx =  12
idx =  6
idx =  21
idx =  15
idx =  21
idx =  14
idx =  3
idx =  2
idx =  1
idx =  21
idx =  18
idx =  24
idx =  7
idx =  25
idx =  6
idx =  25
idx =  18
idx =  10
idx =  16
idx =  2
idx =  3
idx =  8
idx =  15
idx =  12
idx =  11
idx =  7
idx =  1
idx =  12
idx =  10
idx =  2
idx =  7
idx =  7
idx =  11
idx =  7
idx =  24
idx =  12
idx =  16
idx =  11
Sampling:
list of sampled indices: [12, 17, 24, 14, 13, 9, 10, 22, 24, 6, 13, 11, 12, 6, 21, 15, 21, 14, 3, 2, 1, 21, 18, 24, 7, 25, 6, 25, 18, 10, 16, 2, 3, 8, 15, 12, 11, 7, 1, 12, 10, 2, 7, 7, 11, 7, 24, 12, 16, 11, 0]
list of sampled characters: ['l', 'q', 'x', 'n', 'm', 'i', 'j', 'v', 'x', 'f', 'm', 'k', 'l', 'f', 'u', 'o', 'u', 'n', 'c', 'b', 'a', 'u', 'r', 'x', 'g', 'y', 'f', 'y', 'r', 'j', 'p', 'b', 'c', 'h', 'o', 'l', 'k', 'g', 'a', 'l', 'j', 'b', 'g', 'g', 'k', 'g', 'x', 'l', 'p', 'k', '\n']


In [81]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    loss , cache = rnn_forward(X, Y, a_prev, parameters)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    gradients = clip(gradients, 5)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]

In [82]:
np.random.seed(1)
vocab_size, n_a = 27 , 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1) , np.random.randn(vocab_size, 1)
parameters = {'Wax':Wax , 'Waa':Waa, 'Wya':Wya, 'b':b, 'by':by}
X= [12,3,5,11,22,3]
Y= [4,14,11,22,25,26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters)
print('loss = ', loss)
print('gradients[\'dWaa\'][1][2] = ', gradients['dWaa'][1][2])
# print('gradients =', gradients)
# print('gradients[\'dWax\']=', gradients['dWax'])
print('np.argmx(gradients[\'dWax\']) = ', np.argmax(gradients['dWax']))
# print('gradients[\'dWya\'] = ', gradients['dWya'])
print('gradients[\'dWya\'][1][2] = ', gradients['dWya'][1][2]) 
print('gradients[\'db\'][4]=', gradients['db'][4])
print('gradients[\'dby\'][1] = ',gradients['dby'][1])
print('a_last[4] =', a_last[4])

loss =  126.50397572165372
gradients['dWaa'][1][2] =  0.1947093153471366
np.argmx(gradients['dWax']) =  93
gradients['dWya'][1][2] =  -0.007773876032002552
gradients['db'][4]= [-0.06809825]
gradients['dby'][1] =  [0.01538192]
a_last[4] = [-1.]


In [85]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50 , dino_names = 7,vocab_size = 27):
    n_x , n_y = vocab_size, vocab_size
    parameters = initialize_parameters(n_a, n_x, n_y)
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open('dinos.txt') as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
#     examples = examples.lower()
#     examples = examples.strip()
    
    np.random.seed(0)
    np.random.shuffle(examples) #随机
    
    a_prev = np.zeros((n_a, 1))
    
    #循环迭代
    for j in range(num_iterations):
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        Y = X[1:] + [char_to_ix['\n']]
        
        curr_loss , gradients, a_prev = optimize(X, Y, a_prev, parameters)
        loss = smooth(loss, curr_loss) 
        
        if j % 2000 == 0:
            print('Iteration: %d, Loss:%f' % (j, loss) + '\n')
            
            seed = 0
            for name in range(dino_names):
                sampled_indices = sample(parameters , char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed = seed + 1
            
            print('\n')
            
    return parameters

In [88]:
parameters = model(data, ix_to_char, char_to_ix)


Iteration: 0, Loss:23.087336

idx =  14
idx =  11
idx =  26
idx =  24
idx =  23
idx =  20
idx =  4
idx =  13
idx =  6
idx =  17
idx =  15
idx =  5
idx =  25
idx =  8
idx =  19
idx =  17
idx =  23
idx =  1
idx =  19
idx =  10
idx =  11
idx =  10
idx =  22
idx =  21
idx =  0
Nkzxwtdmfqoeyhsqwasjkjvu
idx =  11
idx =  14
idx =  5
idx =  2
idx =  0
Kneb
idx =  11
idx =  26
idx =  24
idx =  23
idx =  20
idx =  4
idx =  13
idx =  6
idx =  17
idx =  15
idx =  5
idx =  25
idx =  8
idx =  19
idx =  17
idx =  23
idx =  1
idx =  19
idx =  10
idx =  11
idx =  10
idx =  22
idx =  21
idx =  0
Kzxwtdmfqoeyhsqwasjkjvu
idx =  14
idx =  5
idx =  2
idx =  0
Neb
idx =  26
idx =  24
idx =  23
idx =  20
idx =  4
idx =  13
idx =  6
idx =  17
idx =  15
idx =  5
idx =  25
idx =  8
idx =  19
idx =  17
idx =  23
idx =  1
idx =  19
idx =  10
idx =  11
idx =  10
idx =  22
idx =  21
idx =  0
Zxwtdmfqoeyhsqwasjkjvu
idx =  5
idx =  2
idx =  0
Eb
idx =  24
idx =  23
idx =  20
idx =  4
idx =  13
idx =  6
idx =  17
idx =

Iteration: 18000, Loss:22.914060

idx =  16
idx =  8
idx =  25
idx =  21
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Phyusaurus
idx =  13
idx =  5
idx =  12
idx =  1
idx =  1
idx =  0
Melaa
idx =  13
idx =  25
idx =  19
idx =  19
idx =  16
idx =  8
idx =  15
idx =  18
idx =  15
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Myssphorosaurus
idx =  16
idx =  5
idx =  7
idx =  1
idx =  12
idx =  15
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Pegalosaurus
idx =  25
idx =  20
idx =  18
idx =  15
idx =  15
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Ytroosaurus
idx =  5
idx =  7
idx =  1
idx =  12
idx =  15
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Egalosaurus
idx =  20
idx =  18
idx =  15
idx =  12
idx =  15
idx =  14
idx =  15
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Trolonosaurus


Iteration: 20000, Loss:22.939454

idx =  14
idx =  12
idx =  25


Iteration: 34000, Loss:22.515899

idx =  13
idx =  1
idx =  21
idx =  19
idx =  9
idx =  18
idx =  1
idx =  16
idx =  15
idx =  0
Mausirapo
idx =  12
idx =  9
idx =  3
idx =  5
idx =  0
Lice
idx =  12
idx =  21
idx =  19
idx =  20
idx =  15
idx =  4
idx =  15
idx =  14
idx =  0
Lustodon
idx =  14
idx =  4
idx =  1
idx =  0
Nda
idx =  25
idx =  19
idx =  8
idx =  9
idx =  3
idx =  8
idx =  5
idx =  18
idx =  15
idx =  7
idx =  21
idx =  2
idx =  15
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Yshicherogubosaurus
idx =  7
idx =  1
idx =  1
idx =  3
idx =  8
idx =  21
idx =  2
idx =  1
idx =  2
idx =  9
idx =  19
idx =  1
idx =  21
idx =  18
idx =  21
idx =  19
idx =  0
Gaachubabisaurus
idx =  20
idx =  18
idx =  15
idx =  4
idx =  15
idx =  14
idx =  8
idx =  15
idx =  12
idx =  21
idx =  19
idx =  1
idx =  21
idx =  18
idx =  14
idx =  9
idx =  18
idx =  1
idx =  17
idx =  21
idx =  18
idx =  1
idx =  14
idx =  20
idx =  9
idx =  20
idx =  1
idx =  14
idx =  0
Tro